In [1]:
import json
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil

from PIL import Image

path = './data/data-of-baselines.json'

In [2]:
#load data from the json file
with open(path) as train_file:
    data = json.load(train_file)

# Survay results:

upload Id workers and search for duplicate workers

In [3]:
raw_df_state_to_action = pd.DataFrame.from_dict(data['humanModel'], orient='index')
raw_df_state_to_action = raw_df_state_to_action.drop(0, axis=1)

participants_df = pd.DataFrame.from_dict(data['all-games'], orient='index')

# path = 'data/'+map_dir+'/workers' # use your path
# all_files = glob.glob(path + "/*.csv")

# li = []
# for filename in all_files:
#     fromOne = pd.read_csv(filename, index_col=None, header=0)
#     fromOne['filename'] = filename
#     li.append(fromOne)

# workers = pd.concat(li, axis=0, ignore_index=True)
# workers = workers.set_index('Answer.surveycode')
# workers.index = workers.index.map(lambda code: '-' + code[:-3])
# # print(workers.index[workers.index.duplicated()])
# workers = workers[~workers.index.duplicated(keep='first')]
# participants_df['WorkerId'] = workers['WorkerId']
# participants_df = participants_df.dropna(subset = ['WorkerId'])
participants_df

,behavior,computer_score,human_score,additional_comments,birth_year,collaborative_value,computer_value,education,gender,predictable_value,selfishly_value,wisely_value
-N2NfRS60c3BmQzlP6_Y,closest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-N2NfTHvrwc1OhLXrvfF,closest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-N2NfUaQ0uIclQzqI9Jv,closest,9.000,10.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-N2NfbswCIxy_yZjwffA,closest,12.000,11.000,good,1930,6,5,Master's degree,female,5,7,5


In [4]:
participants_df.columns

Index(['behavior', 'computer_score', 'human_score', 'additional_comments',
       'birth_year', 'collaborative_value', 'computer_value', 'education',
       'gender', 'predictable_value', 'selfishly_value', 'wisely_value'],
      dtype='object')

In [5]:
# index_to_drop = participants_df[participants_df.duplicated('WorkerId', keep='first')].index
# print(len(index_to_drop))

# participants_df = participants_df[~participants_df.duplicated('WorkerId', keep='first')]

number of people that answer the survay only once:

In [6]:
len(participants_df)

4

In [7]:
# participants_df[['computer_score', 'human_score', 'behavior']].to_excel("agents_score.xlsx")

In [8]:
participants_df[['additional_comments', 'behavior']][participants_df['additional_comments'].notna()][participants_df['additional_comments'] != ""].tail()

c:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,additional_comments,behavior
-N2NfbswCIxy_yZjwffA,good,closest


In [9]:
participants_df.loc[: ,'human_score'] = pd.to_numeric(participants_df['human_score'], errors='coerce')
participants_df.loc[: ,'computer_score'] = pd.to_numeric(participants_df['computer_score'], errors='coerce')
participants_df.loc[: ,'collaborative_value'] = pd.to_numeric(participants_df['collaborative_value'], errors='coerce')
participants_df.loc[: ,'predictable_value'] = pd.to_numeric(participants_df['predictable_value'], errors='coerce')
participants_df.loc[: ,'selfishly_value'] = pd.to_numeric(participants_df['selfishly_value'], errors='coerce')
participants_df.loc[: ,'wisely_value'] = pd.to_numeric(participants_df['wisely_value'], errors='coerce')
participants_df.loc[: ,'computer_value'] = pd.to_numeric(participants_df['computer_value'], errors='coerce')

In [10]:
participants_df.groupby('behavior')[['computer_score', 'human_score', 'collaborative_value', 'predictable_value', 'selfishly_value', 'wisely_value', 'computer_value']].mean()

,computer_score,human_score,collaborative_value,predictable_value,selfishly_value,wisely_value,computer_value
behavior,,,,,,,
closest,10.5,10.5,6.0,5.0,7.0,5.0,5.0


In [11]:
participants_df['behavior'].value_counts()

closest    4
Name: behavior, dtype: int64

In [12]:
participants_df['gender'].value_counts()

female    1
Name: gender, dtype: int64

In [13]:
participants_df['education'].value_counts()

Master's degree    1
Name: education, dtype: int64

# View some data

In [14]:
# remove from the dataset :
# raw_df_state_to_action = raw_df_state_to_action.drop(index_to_drop)
raw_df_state_to_action.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, -N2NfTHvrwc1OhLXrvfF to -N2NfbswCIxy_yZjwffA
Columns: 30 entries, 1 to 30
dtypes: object(30)
memory usage: 744.0+ bytes


In [15]:
def extractAction(cell):
    if cell != None:
        return int(cell['humanAction'])
    return np.nan

def extractState(cell):
    if cell == None:
        return cell
    computer_pos = [cell['stateCoords'][0], cell['stateCoords'][1]]
    human_pos = [cell['stateCoords'][2], cell['stateCoords'][3]]
    stag_pos = [cell['stateCoords'][4], cell['stateCoords'][5]]

    shrubs_pos = [cell['stateCoords'][6], cell['stateCoords'][7], cell['stateCoords'][8], cell['stateCoords'][9]]
    
    return [computer_pos, human_pos, stag_pos, shrubs_pos]
    
state_df = pd.DataFrame(columns=raw_df_state_to_action.columns)
action_df = pd.DataFrame(columns=raw_df_state_to_action.columns)
for col in raw_df_state_to_action:
    state_df[col] = raw_df_state_to_action[col].apply(extractState)
    action_df[col] = raw_df_state_to_action[col].apply(extractAction)

In [16]:
state_df.head()

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
-N2NfTHvrwc1OhLXrvfF,"[[0, 0], [0, 4], [2, 2], [2, 0, 2, 4]]","[[1, 0], [1, 4], [1, 2], [2, 0, 2, 4]]",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
-N2NfUaQ0uIclQzqI9Jv,"[[0, 0], [0, 4], [2, 2], [0, 2, 4, 2]]","[[0, 1], [0, 3], [1, 2], [0, 2, 4, 2]]","[[0, 2], [0, 2], [1, 1], [0, 4, 4, 2]]","[[0, 3], [1, 2], [2, 1], [0, 4, 4, 2]]","[[0, 4], [2, 2], [2, 1], [3, 1, 4, 2]]","[[1, 4], [3, 2], [1, 1], [3, 1, 4, 2]]","[[2, 4], [4, 2], [2, 1], [3, 1, 4, 3]]","[[3, 4], [4, 1], [2, 0], [3, 1, 4, 3]]","[[4, 4], [4, 2], [3, 0], [3, 1, 4, 3]]","[[4, 3], [3, 2], [3, 1], [3, 1, 0, 3]]",...,"[[0, 0], [1, 1], [1, 2], [4, 4, 3, 0]]","[[1, 0], [2, 1], [1, 1], [4, 4, 3, 0]]","[[2, 0], [3, 1], [1, 2], [4, 4, 3, 0]]","[[3, 0], [3, 0], [1, 3], [4, 4, 4, 2]]","[[4, 0], [4, 0], [0, 3], [4, 4, 4, 2]]","[[4, 1], [4, 1], [0, 4], [4, 4, 4, 2]]","[[4, 2], [4, 2], [1, 4], [4, 4, 1, 2]]","[[4, 3], [4, 3], [0, 4], [4, 4, 1, 2]]","[[4, 4], [4, 4], [1, 4], [0, 1, 1, 2]]","[[3, 4], [3, 4], [2, 4], [0, 1, 1, 2]]"
-N2NfbswCIxy_yZjwffA,"[[0, 0], [0, 4], [2, 2], [1, 4, 0, 3]]","[[0, 1], [1, 4], [1, 2], [0, 4, 0, 3]]","[[0, 2], [0, 4], [1, 1], [2, 4, 0, 3]]","[[0, 3], [0, 3], [2, 1], [2, 4, 4, 1]]","[[1, 3], [1, 3], [3, 1], [2, 4, 4, 1]]","[[2, 3], [2, 3], [2, 1], [2, 4, 4, 1]]","[[2, 4], [3, 3], [3, 1], [4, 0, 4, 1]]","[[3, 4], [3, 2], [4, 1], [4, 0, 4, 1]]","[[4, 4], [4, 2], [4, 2], [4, 0, 4, 1]]","[[4, 3], [4, 1], [4, 3], [4, 0, 1, 2]]",...,"[[1, 3], [2, 2], [1, 2], [4, 2, 0, 2]]","[[2, 3], [2, 3], [0, 2], [4, 2, 0, 2]]","[[1, 3], [3, 3], [1, 2], [4, 2, 0, 2]]","[[0, 3], [4, 3], [2, 2], [4, 2, 0, 2]]","[[0, 2], [4, 2], [2, 3], [0, 0, 0, 1]]","[[0, 1], [4, 3], [2, 4], [0, 0, 0, 4]]","[[0, 0], [4, 4], [3, 4], [4, 0, 0, 4]]","[[1, 0], [3, 4], [2, 4], [4, 0, 0, 4]]","[[0, 0], [2, 4], [3, 4], [4, 0, 0, 4]]","[[1, 0], [1, 4], [4, 4], [4, 0, 0, 4]]"


# view one game

In [17]:
from sys import stdout

A_AGENT = 0  # base
B_AGENT = 1

STAG = 2  # hunt
PLANT = 3

def print_matrix(obs, grid_size):
    matrix = np.full((grid_size[0], grid_size[1], 4), False, dtype=bool)

    a, b, stag = (obs[0], obs[1]), (obs[2], obs[3]), (obs[4], obs[5])
    matrix[a[0]][a[1]][A_AGENT] = True
    matrix[b[0]][b[1]][B_AGENT] = True
    matrix[stag[0]][stag[1]][STAG] = True
    for i in range(6, len(obs), 2):
        plant = obs[i], obs[i + 1]
        matrix[plant[0]][plant[1]][PLANT] = True

    symbols = ("S", "P")

    stdout.write("╔════════════════════════════╗\n")
    for row in matrix:
        stdout.write("║ ·")
        for col in row:
            cell = []
            cell.append("A") if col[0] == 1 else cell.append(" ")
            cell.append("B") if col[1] == 1 else cell.append(" ")
            cell.append(symbols[0]) if col[2] == 1 else cell.append(" ")
            cell.append(symbols[1]) if col[3] == 1 else cell.append(" ")
            stdout.write("".join(cell) + "·")
        stdout.write(" ║")
        stdout.write("\n")
    stdout.write("╚════════════════════════════╝\n\r")
    stdout.flush()

In [18]:
def flatten(t):
    return [item for sublist in t for item in sublist]

def render(row):
    row_s = state_df.loc[row, :]
    row_a = action_df.loc[row, :]
    cols_i = row_a.count()
    for i in range(1, cols_i+1):
        state = row_s.at[i]
        print_matrix(flatten(state), [5, 5])
        print(row_a[i])
    

In [21]:
render('-N2NfbswCIxy_yZjwffA')

╔════════════════════════════╗
║ ·A   ·    ·    ·   P· B  · ║
║ ·    ·    ·    ·    ·   P· ║
║ ·    ·    ·  S ·    ·    · ║
║ ·    ·    ·    ·    ·    · ║
║ ·    ·    ·    ·    ·    · ║
╚════════════════════════════╝
40.0
╔════════════════════════════╗
║ ·    ·A   ·    ·   P·   P· ║
║ ·    ·    ·  S ·    · B  · ║
║ ·    ·    ·    ·    ·    · ║
║ ·    ·    ·    ·    ·    · ║
║ ·    ·    ·    ·    ·    · ║
╚════════════════════════════╝
38.0
╔════════════════════════════╗
║ ·    ·    ·A   ·   P· B  · ║
║ ·    ·  S ·    ·    ·    · ║
║ ·    ·    ·    ·    ·   P· ║
║ ·    ·    ·    ·    ·    · ║
║ ·    ·    ·    ·    ·    · ║
╚════════════════════════════╝
37.0
╔════════════════════════════╗
║ ·    ·    ·    ·AB  ·    · ║
║ ·    ·    ·    ·    ·    · ║
║ ·    ·  S ·    ·    ·   P· ║
║ ·    ·    ·    ·    ·    · ║
║ ·    ·   P·    ·    ·    · ║
╚════════════════════════════╝
40.0
╔════════════════════════════╗
║ ·    ·    ·    ·    ·    · ║
║ ·    ·    ·    ·AB  ·    · ║
║ ·    ·    ·    · 

In [20]:
action_df[(action_df.count(axis=1) > 5)]

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
-N2NfUaQ0uIclQzqI9Jv,37,37,40.0,40.0,40.0,40.0,37.0,39.0,38.0,38.0,...,40.0,40.0,37.0,40.0,39.0,39.0,39.0,39.0,38.0,38.0
-N2NfbswCIxy_yZjwffA,40,38,37.0,40.0,40.0,40.0,37.0,40.0,37.0,37.0,...,39.0,40.0,40.0,37.0,39.0,39.0,38.0,38.0,38.0,38.0


In [ ]:
action_df[len(action_df.columns) + 1] = np.NaN
action_df.tail()

# Make Dataset

In [ ]:
dir_path = './data/humanModel_v0_dataset'
if os.path.exists(dir_path):
    shutil.rmtree(dir_path)
os.mkdir(dir_path)
for action in ['32','37','38','39','40']:
    os.mkdir(dir_path+'/'+action)

counter = 0
for (idxRow, s1), (_, s2) in zip(state_df.iterrows(), action_df.iterrows()):
    for (idxCol, state), (_, action) in zip(s1.iteritems(), s2.iteritems()):
        # check if it is not the last state
        # the last state not enter our model, since it is an end state that not contains any dirts and not preform an action
        if not np.isnan(action_df.loc[idxRow, idxCol+1]):
            im = Image.fromarray((state * 255).astype(np.uint8))
            path = f'data/humanModel_v0_dataset/{int(action)}/{idxRow}_{idxCol}.png'
            if counter % 500 == 0:
                # print every 500 saved images
                print(f'{idxRow}_{idxCol}.png saved! at action {action}')
            counter += 1
            im.save(path)
        else:
            break
#         print (state, action, idxCol, idxRow)
print(f'{counter} images have been saved')

In [ ]:
rowToImage('-N2IDOVbGpY7VfpiuPB-')

In [ ]:
# index = "-MqplpytsnX0zCv_KW9i"
# col = 3
# plt.imshow(state_df.loc[index, col])
# title = "id: " + index + ", col: " + str(col) + ", action: " + str(action_df.loc[index, col])
# plt.title(title)
# plt.show()